In [1]:
import calendar
import numpy as np
import xarray as xr
import pandas as pd
import os
 
from datetime import timedelta, date

Get time series of 3 locations from PRISM dataset

In [2]:
# casper_2020
# extract daily prec. Tmax and Tmin from PRISM dataset (closest/nearest grid)

Washington   45.5470° N, 123.1386° W
Multnomah    45.5146° N, 122.5863° W
Clackamas    45.4076° N, 122.5704° W

In [3]:
def daterange(date1, date2):
    for n in range(int ((date2 - date1).days)+1):
        yield date1 + timedelta(n)
         

In [4]:
n_loc = 3

lat = np.empty(n_loc)
lon = np.empty(n_loc)

lat[0] =   45.5470
lon[0] = -123.1386
lat[1] =   45.5146
lon[1] = -122.5863
lat[2] =   45.4076
lon[2] = -122.5704

# create an 8-bit signed integer ( represents integers from -128 to 127) array
ii = np.empty(n_loc, dtype=np.int8)
jj = np.empty(n_loc, dtype=np.int8)

year_s = 1981
year_e = 2021
#year_e = 1981

dir_i = '/glade/campaign/mmm/c3we/prein/observations/PRISM/data/'


In [5]:
#Calculate number of days between two dates
date_s = date(year_s, 1, 1)
date_e = date(year_e, 12, 31)
delta = date_e - date_s
n_day = delta.days + 1

# create a range of dates list  
yyyymmdd = [dt.strftime("%Y-%m-%d") for dt in daterange(date_s, date_e)]

# and then convert it to datetime      
yyyymmdd = pd.to_datetime(yyyymmdd)


seasons = ['DJF', 'DJF', 'MAM', 'MAM', 'MAM', 'JJA', 'JJA', 'JJA', 'SON', 'SON', 'SON', 'DJF']
#Create a dictionary that maps a month to a season:
month_to_season = dict(zip(range(1,13), seasons))
 
season = yyyymmdd.month.map(month_to_season)

# creat a 2D numpy array [n_day, n_loc]
arr_2d = np.array([[0]*n_loc]*n_day, dtype=np.float32)


In [6]:
# Find the nearest latitude and longitude grid for a point
#  (45.54,-123.1), (45.5,-122.6), (45.42, -122.6)
with xr.open_dataset('/glade/campaign/mmm/c3we/prein/observations/PRISM/data/PR/PRISM_daily_ppt_1981.nc') as ds_prism:
        lat_2d = ds_prism.lat.values
        lon_2d = ds_prism.lon.values
        
        for count in range(0, n_loc):
            a = np.abs(lat_2d-lat[count] ) + np.abs(lon_2d-lon[count] )
            ii[count],jj[count] = np.unravel_index(a.argmin(), a.shape)

var_s = "PR"
#var_s = "Tmax"
#var_s = "Tmin"

ct_s = 0
for year in range(year_s, year_e+1, 1):
    
    if var_s == 'PR':
        flnm_prism = dir_i + var_s + '/PRISM_daily_ppt_'+ str(year) + '.nc'
    else:    
         flnm_prism = dir_i + var_s + '/PRISM_daily_' + var_s.lower() + '_'+ str(year) + '.nc'
    with xr.open_dataset(flnm_prism ) as ds_prism:
        print(flnm_prism)
        for count in range(0, n_loc):
            # get variable names
            data_3d = ds_prism.data_vars 
            
            data_1d = data_3d[var_s][:, ii[count],jj[count]]
            if count == 0:
                ct_e = ct_s + len(data_1d) 
            
            arr_2d[ct_s:ct_e,count] = data_1d
            
        #print(ct_s,ct_e)
        ct_s = ct_e 

flnm_o = '/glade/campaign/mmm/c3we/INNOVATOR/' + var_s + '_1981-2021.csv'        
print(flnm_o)        


data = {'date':yyyymmdd, 'year':yyyymmdd.year, 'season':yyyymmdd.month.map(month_to_season),'Washington':arr_2d[:,0], 'Multnomah':arr_2d[:,1], 'Clackamas':arr_2d[:,2]}
df = pd.DataFrame(data) 

df.to_csv(flnm_o,index=False)  
 


/glade/campaign/mmm/c3we/prein/observations/PRISM/data/PR/PRISM_daily_ppt_1981.nc
/glade/campaign/mmm/c3we/prein/observations/PRISM/data/PR/PRISM_daily_ppt_1982.nc
/glade/campaign/mmm/c3we/prein/observations/PRISM/data/PR/PRISM_daily_ppt_1983.nc
/glade/campaign/mmm/c3we/prein/observations/PRISM/data/PR/PRISM_daily_ppt_1984.nc
/glade/campaign/mmm/c3we/prein/observations/PRISM/data/PR/PRISM_daily_ppt_1985.nc
/glade/campaign/mmm/c3we/prein/observations/PRISM/data/PR/PRISM_daily_ppt_1986.nc
/glade/campaign/mmm/c3we/prein/observations/PRISM/data/PR/PRISM_daily_ppt_1987.nc
/glade/campaign/mmm/c3we/prein/observations/PRISM/data/PR/PRISM_daily_ppt_1988.nc
/glade/campaign/mmm/c3we/prein/observations/PRISM/data/PR/PRISM_daily_ppt_1989.nc
/glade/campaign/mmm/c3we/prein/observations/PRISM/data/PR/PRISM_daily_ppt_1990.nc
/glade/campaign/mmm/c3we/prein/observations/PRISM/data/PR/PRISM_daily_ppt_1991.nc
/glade/campaign/mmm/c3we/prein/observations/PRISM/data/PR/PRISM_daily_ppt_1992.nc
/glade/campaign/

####### finished

In [ ]:
#original write csv file
flnm_o = '/glade/campaign/mmm/c3we/INNOVATOR/' + var_s + '_1981-2021.csv'        
print(flnm_o)        
np.savetxt(flnm_o, arr_2d,  fmt='%.3f', delimiter=',')
 
#add a header    
cmd_sed = 'sed -i "1i Washington,Multnomah,Clackamas" flnm_o'    
cmd_sed = cmd_sed.replace("flnm_o", flnm_o)
print(cmd_sed)
os.system(cmd_sed)

In [16]:
# the same as previous " the nearest latitude and longitude grid "

ct_s = 0
for year in range(year_s, year_e+1, 1):
    flnm_prism = dir_i + 'PR/PRISM_daily_ppt_'+ str(year) + '.nc'
    with xr.open_dataset(flnm_prism ) as ds_prism:
        print(flnm_prism)
        for count in range(0, n_loc):
            data_1d = ds_prism.PR.sel(rlat=lat[count], rlon=lon[count], method='nearest')
            
            if count == 0:
                ct_e = ct_s + len(data_1d) 
            
            arr_2d[ct_s:ct_e,count] = data_1d
            
        #print(ct_s,ct_e)
        ct_s = ct_e 

np.savetxt('pr_1981-2021.csv', arr_2d,  fmt='%.3f', delimiter=',')
 
# add a header 
os.system("sed -i '1i Washington,Multnomah,Clackamas' pr_1981-2021.csv") 
 

/glade/campaign/mmm/c3we/prein/observations/PRISM/data/PR/PRISM_daily_ppt_1981.nc


0